In [1]:
###############################
# Using Phantasy              #
###############################

In [2]:
import phantasy
import matplotlib.pyplot as plt
import labframePlotFun
%matplotlib inline

from epics import caget, caput, cainfo

In [3]:
# e.g. machine-name: 'FRIB_LEBT', segment: 'LEBT' (or skipped)
mp = phantasy.MachinePortal(machine='/projects/rea3/data/CrispAnalysis/phantasy-machines/ReA', segment='GPL_VA')

[2020-04-02T16:25:37.530] INFO: phantasy.library.operation.core: Load new machine: 'ReA', segment: 'GPL_VA'
[2020-04-02T16:25:37.531] INFO: phantasy.library.parser.config: Importing '/projects/rea3/data/CrispAnalysis/phantasy-machines/ReA' from '/projects/rea3/data/CrispAnalysis/phantasy-machines/ReA'
[2020-04-02T16:25:37.533] INFO: phantasy.library.parser.config: Loading machine configuration file: /projects/rea3/data/CrispAnalysis/phantasy-machines/ReA/phantasy.ini
[2020-04-02T16:25:37.534] INFO: phantasy.library.operation.lattice: Loading segment: 'GPL_VA'
[2020-04-02T16:25:37.537] WARNING: phantasy.library.operation.lattice: Default UNICORN policy will be applied.
[2020-04-02T16:25:37.538] INFO: phantasy.library.operation.lattice: Loading PV data from CSV/SQLite: /projects/rea3/data/CrispAnalysis/phantasy-machines/ReA/GPL/channels_gpl.csv
[2020-04-02T16:25:37.540] INFO: phantasy.library.pv.datasource: Set source to be CSV table: channels_gpl.csv.
[2020-04-02T16:25:37.586] INFO: pha

In [ ]:
# Make lat object?
lat = mp.work_lattice_conf

In [6]:
#
# Synchronizing values
#

# sync settings from accelerator (VA or REAL)
#lat.sync_settings(data_source='control')

# sync settings from model to accelerator
#lat.sync_settings(data_source='model')

In [9]:
#
# Spit out current values
#

mp.get_all_types()
#elem = mp.get_elements(type=['SOL', 'HCOR', 'VCOR', 'CAV', 'BEND', 'QUAD'])
elem = mp.get_elements(type=['SOL', 'HCOR', 'VCOR', 'QUAD'])
#mp.get_pv_values(elem, ['I'])
data_0 = mp.get_pv_values(elem)

In [11]:
#
# Set Drive limits (not cavities)
#

from epics import caget, caput, cainfo

# get all element names, remove FC/VD, then CAV too
names = mp.get_all_names()
names = [ name for name in names if ":FC_" not in name and ":VD_" not in name ]
names = [name for name in  names if ":CAV_" not in name ]

print("Process Variable\t\t\tDRVL\tDRVH")
for name in names:
    pv_va = "VA:" + name + ":I_CSET"
    if ":DCV_" in pv_va: pv_va = pv_va.replace(":DCV_", ":PSC1_")
    if ":DCH_" in pv_va: pv_va = pv_va.replace(":DCH_", ":PSC2_")
    if "BTS26:PSC1" in pv_va: pv_va = pv_va.replace("BTS26:PSC1", "BTS30:PSC1")
    if "BTS26:PSC2" in pv_va: pv_va = pv_va.replace("BTS26:PSC2", "BTS30:PSC2")
    
    pv = name + ":I_CSET"
    
    drvl = caget(pv + ".DRVL")
    drvh = caget(pv + ".DRVH")
    #print(pv + ",\t\t" + str(drvl) + ",\t" + str(drvh))
    
    caput(pv_va + ".DRVL", drvl)
    drvl_va = caget(pv_va + ".DRVL")
    
    caput(pv_va + ".DRVH", drvh)
    drvh_va = caget(pv_va + ".DRVH")
    
    #print(pv_va + ",\t\t" + str(drvl_va) + ",\t" + str(drvh_va))
    
    #value = caget(pv)
    #caput(pv_va,value)



Process Variable			DRVL	DRVH
cannot connect to VA:REA_BTS32:DH_D1376:I_CSET.DRVL
cannot connect to VA:REA_BTS32:DH_D1376:I_CSET.DRVL
cannot connect to VA:REA_BTS32:DH_D1376:I_CSET.DRVH
cannot connect to VA:REA_BTS32:DH_D1376:I_CSET.DRVH


In [12]:
#
# Set Drive limits (cavities only)
#

# get all element names, exclude everything except CAV, then replace CAV with RFC
names = mp.get_all_names()
names = [name for name in  names if ":CAV_" in name ]
names = [name.replace(":CAV_", ":RFC_") for name in names ]

# get drive limits from live machine, apply to VA
for name in names:
    pv_amp = name + ":E_CSET"
    pv_pha = name + ":PHA_CSET"
    
    drvh = caget(pv_amp + ".DRVH")
    caput("VA:" + pv_amp + ".DRVH", drvh)
    
    caput("VA:" + pv_pha + ".DRVL", -360)
    caput("VA:" + pv_pha + ".DRVH", 360)
    
    #val_amp = caget(pv_amp)
    #val_pha = caget(pv_pha)
    
    #caput("VA:" + pv_amp, val_amp)
    #caput("VA:" + pv_pha, val_pha)